In [4]:
import requests

# Caudoputamen connectivity data
caudoputamenInputUrl = "https://bams2.bams1.org/search/connections/inputs/?page={number}&query=Caudoputamen+~+%28CP%29"
caudoputamenOutputUrl = "https://bams2.bams1.org/search/connections/outputs/?page={number}&query=Caudoputamen+~+%28CP%29"

def write_stuff(url, replace_str, numbers, output_file):
    with open(output_file, 'wb') as file:
        file.write(b'')
    print("Getting", url)
    for i in range(1, numbers+1):
        print("."*i)
        url = url.replace(replace_str, str(i))
        res = requests.get(url)
        
        with open(output_file, 'ab') as file:
            file.write(res.content)
    
write_stuff(caudoputamenInputUrl, "{numbers}", 17, 'BAMS_data/connectivity/caudoputamen_inputs.html')
write_stuff(caudoputamenOutputUrl, "{numbers}", 3, 'BAMS_data/connectivity/caudoputamen_outputs.html')



Getting https://bams2.bams1.org/search/connections/inputs/?page={number}&query=Caudoputamen+~+%28CP%29
.
..
...
....
.....
......
.......
........
.........
..........
...........
............
.............
..............
...............
................
.................
Getting https://bams2.bams1.org/search/connections/outputs/?page={number}&query=Caudoputamen+~+%28CP%29
.
..
...


In [8]:
from bs4 import BeautifulSoup
import csv
import re

def write_to_csv(input_html, output_csv):
    htmls = []
    with open(input_html, 'r') as file:
            htmls = file.read().split("<!DOCTYPE")

    htmls = list(filter(len, htmls))

    with open(output_csv, 'w') as csvfile:
        csvfile.write("")

    add_header = True    

    output_rows = []
    for html in htmls:
        soup = BeautifulSoup(html)
        table = soup.find("table")


        for table_row in table.findAll('tr'):
            if add_header:
                headers = table_row.findAll('th')
                output_row = []
                for header in headers:
                    output_row.append(header.text.replace("\n", "").strip())
                output_rows.append(output_row)
                add_header = False

            columns = table_row.findAll('td')
            output_row = []
            for column in columns:
                column_text = column.text.replace("\n", "").strip()
                column_text = re.sub(' +', ' ', column_text)
                output_row.append(column_text)
            if len(output_row) > 0:
                output_rows.append(output_row)

    with open(output_csv, 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(output_rows)

    print("Tables from ", input_html, "converted to csv in", output_csv)
    
caud_inputs_html = "BAMS_data/connectivity/caudoputamen_inputs.html"
caud_inp_csv = "BAMS_data/connectivity/caudoputamen_inputs.csv"

caud_outputs_html = "BAMS_data/connectivity/caudoputamen_outputs.html"
caud_out_csv = "BAMS_data/connectivity/caudoputamen_outputs.csv"

write_to_csv(caud_inputs_html, caud_inp_csv)
write_to_csv(caud_outputs_html, caud_out_csv)
    

Tables from  BAMS_data/connectivity/caudoputamen_inputs.html converted to csv in BAMS_data/connectivity/caudoputamen_inputs.csv
Tables from  BAMS_data/connectivity/caudoputamen_outputs.html converted to csv in BAMS_data/connectivity/caudoputamen_outputs.csv
